In [1]:
from glob import glob
import os
import cv2
import matplotlib.pyplot as plt
import albumentations as A
import pandas as pd
from copy import deepcopy

from convert2yolo import convert_to_yolo

df = pd.read_csv("./code2idx.csv")

root = "/home/j-j10b102/166.약품식별_인공지능_개발을_위한_경구약제_이미지_데이터/01.데이터/1.Training/{}/단일경구약제_5000종/"

In [2]:
df[df["item_seq"] == 199303108]

,item_seq,idx,drug_n,name,group
2515,199303108,2515,K-004378,타이레놀정500mg,TS_3_단일


In [5]:
class Albumentation:
    def __init__(self, item_seq):
        self.item_seq = item_seq
        self.item = df[df["item_seq"] == item_seq]
        print("item: ",self.item)

        group_folders = []

        image_group_folder = self.item["group"].iloc[0]


        label_group_folder = image_group_folder.split("_")

        label_group_folder[0] = "TL"

        label_group_folder = "_".join(label_group_folder)

        group_folders.append(image_group_folder)

        group_folders.append(label_group_folder)



        print("group_folders: ", group_folders)
        import random

        arr = {}

        for idx, group_folder in enumerate(group_folders):

            drug_n_folder = self.item["drug_n"].iloc[-1]
            data = {}
            if idx == 0:
                data["drug_n_folder"] = drug_n_folder
                data["path"] = "원천데이터"
                data["extension"] = "png"
                data["dtype"] = "images"

            else:
                data["drug_n_folder"] = drug_n_folder+"_json"
                data["path"] = "라벨링데이터"
                data["extension"] = "json"
                data["dtype"] = "labels"
            data["group_folder"] = group_folder
            path = root.format(data["path"])
            path = path+data["group_folder"]+"/"+data["drug_n_folder"]
            files = [os.path.splitext(file)[0].split("/")[-1] for file in glob(path+"/*") if os.path.isfile(file)]

            files.sort()

            data["files"] = files
            arr[idx] = data

        random.shuffle(arr[0]["files"])



        arr[1]["files"] = deepcopy(arr[0]["files"])
        self.arr = arr
        
    def train_validation_split(self):
        import shutil
        for key, data in self.arr.items():
            path = root.format(data["path"])
            path = path+data["group_folder"]+"/"+data["drug_n_folder"]

            train_folder = path+"/"+data["dtype"]+"/train"
            val_folder = path+"/"+data["dtype"]+"/val"

            if not os.path.exists(train_folder):
                os.makedirs(train_folder)

            if not os.path.exists(val_folder):
                os.makedirs(val_folder)

            files = data["files"]

            total_size = len(files)
            train_size = int(0.7 * total_size)

            trains = files[:train_size]
            vals = files[train_size:]


            if key == 0:

                for filename in trains:
                    src_file = path + "/"+filename+"."+data["extension"]

                    dst_file = train_folder+"/"+filename+"."+data["extension"]

                    shutil.copy(src_file, dst_file)

                for filename in vals:
                    src_file = path + "/"+filename+"."+data["extension"]


                    dst_file = val_folder+"/"+filename+"."+data["extension"]

                    shutil.copy(src_file,dst_file)


            else:

                import json


                for filename in trains:
                    src_file = path + "/"+filename+"."+data["extension"]
                    with open(src_file, "rb") as f:
                        json_data = json.load(f)
                    idx = self.item["idx"].iloc[0]

                    height = json_data["images"][0]["height"]
                    width = json_data["images"][0]["width"]

                    x,y,w,h = json_data["annotations"][0]["bbox"]

                    center_x, center_y, normalized_w, normalized_h = convert_to_yolo(x,y,w,h,width, height)

    #                 filename, _ = os.path.splitext(src_file)
    #                 filename = filename.split("/")[-1]

                    dst_file = train_folder+"/"+filename+".txt"



                    with open(dst_file, "w") as f:
                        f.write(str(idx)+" ")
                        f.write(str(center_x)+" ")
                        f.write(str(center_y)+" ")
                        f.write(str(normalized_w)+" ")
                        f.write(str(normalized_h)+" ")

                for filename in vals:
                    src_file = path + "/"+filename+"."+data["extension"]
                    with open(src_file, "rb") as f:
                        json_data = json.load(f)

                    idx = self.item["idx"].iloc[0]

                    height = json_data["images"][0]["height"]
                    width = json_data["images"][0]["width"]

                    x,y,w,h = json_data["annotations"][0]["bbox"]

                    center_x, center_y, normalized_w, normalized_h = convert_to_yolo(x,y,w,h,width, height)

    #                 filename, _ = os.path.splitext(src_file)
    #                 filename = filename.split("/")[-1]

                    dst_file = val_folder+"/"+filename+".txt"

                    with open(dst_file, "w") as f:
                        f.write(str(idx)+" ")
                        f.write(str(center_x)+" ")
                        f.write(str(center_y)+" ")
                        f.write(str(normalized_w)+" ")
                        f.write(str(normalized_h)+" ")
    
    def augmentation(self, transform, albumentation):
        import json
        img_data = self.arr[0]
        label_data = self.arr[1]
        
        img_path = root.format(img_data["path"])
        img_path = img_path + img_data["group_folder"] +"/"+img_data["drug_n_folder"]
        

        
        
        img_train_folder = img_path +"/"+img_data["dtype"]+"/train"
        img_val_folder = img_path +"/"+img_data["dtype"]+"/val"
        
        if not os.path.exists(img_train_folder):
            os.makedirs(img_train_folder)

        if not os.path.exists(img_val_folder):
            os.makedirs(img_val_folder)
            
        label_path = root.format(label_data["path"])
        label_path = label_path + label_data["group_folder"] + "/"+ label_data["drug_n_folder"]
            
        label_train_folder = label_path + "/"+label_data["dtype"]+"/train"
        label_val_folder = label_path+"/"+label_data["dtype"]+"/val"
        
        if not os.path.exists(label_train_folder):
            os.makedirs(label_train_folder)
            
        if not os.path.exists(label_val_folder):
            os.makedirs(label_val_folder)
            
            
        
        
        
        img_files = img_data["files"]
        
        total_size = len(img_files)
        train_size = int(0.7 * total_size)
        
        img_trains = img_files[:]
        img_vals = img_files[train_size:]
        
        for filename in img_trains:
            img_file = img_path +"/"+filename+"."+img_data["extension"]
            label_file = label_path +"/"+filename+"."+label_data["extension"]
            with open(label_file, "rb") as f:
                json_data = json.load(f)
                
            idx = self.item["idx"].iloc[0]
            
            height = json_data["images"][0]["height"]
            width = json_data["images"][0]["width"]
            
            x,y,w,h = json_data["annotations"][0]["bbox"]
            
            cx,cy,normalized_w,normalized_h = convert_to_yolo(x,y,w,h,width, height)
            bboxes = [[cx,cy,normalized_w, normalized_h]]
            image = cv2.imread(img_file)
            transformed = transform(image=image, bboxes=bboxes, class_labels=[0])
            transformed_img = transformed["image"]
            transformed_bbox = [coord for bbox in transformed["bboxes"] for coord in bbox]
            cx,cy,normalized_w,normalized_h = transformed_bbox
            dst_img_file = img_train_folder +"/"+filename+albumentation+"."+img_data["extension"]
            cv2.imwrite(dst_img_file, transformed_img)

            dst_label_file = label_train_folder+"/"+filename+albumentation+".txt"
#             print(transformed_bbox)
            
            
            with open(dst_label_file, "w") as f:
                f.write(str(idx)+" ")
                f.write(str(cx)+" ")
                f.write(str(cy)+" ")
                f.write(str(normalized_w)+" ")
                f.write(str(normalized_h)+" ")
                
        for filename in img_vals:
            img_file = img_path +"/"+filename+"."+img_data["extension"]
            label_file = label_path +"/"+filename+"."+label_data["extension"]
            with open(label_file, "rb") as f:
                json_data = json.load(f)
                
            idx = self.item["idx"].iloc[0]
            
            height = json_data["images"][0]["height"]
            width = json_data["images"][0]["width"]
            
            x,y,w,h = json_data["annotations"][0]["bbox"]
            
            cx,cy,normalized_w,normalized_h = convert_to_yolo(x,y,w,h,width, height)
            bboxes = [[cx,cy,normalized_w, normalized_h]]
            image = cv2.imread(img_file)
            transformed = transform(image=image, bboxes=bboxes, class_labels=[0])
            transformed_img = transformed["image"]
            transformed_bbox = [coord for bbox in transformed["bboxes"] for coord in bbox]
            cx,cy,normalized_w,normalized_h = transformed_bbox
            dst_img_file = img_val_folder +"/"+filename+albumentation+"."+img_data["extension"]
            cv2.imwrite(dst_img_file, transformed_img)

            dst_label_file = label_val_folder+"/"+filename+albumentation+".txt"
#             print(transformed_bbox)
            
            
            with open(dst_label_file, "w") as f:
                f.write(str(idx)+" ")
                f.write(str(cx)+" ")
                f.write(str(cy)+" ")
                f.write(str(normalized_w)+" ")
                f.write(str(normalized_h)+" ")
   
        
    def flip(self):
        
        transform = A.Compose([
            A.VerticalFlip(p=0.7),
            A.HorizontalFlip(p=0.7)
        ], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))
        
        albumentation = "_verticalflip"
        
        self.augmentation(transform, albumentation)
        
    def brightness(self):
        transform = A.Compose([
            A.RandomBrightnessContrast(brightness_limit=(-0.2, -0.2), p=1.0)
        ], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))
        
        albumentation = "_brightness"
        self.augmentation(transform, albumentation)
      
                
    
    def getFolderCount(self):
    
        img_data = self.arr[0]
        label_data = self.arr[1]
        
        img_path = root.format(img_data["path"])
        img_path = img_path + img_data["group_folder"] +"/"+img_data["drug_n_folder"]
        
        label_path = root.format(label_data["path"])
        label_path = label_path + label_data["group_folder"] + "/"+ label_data["drug_n_folder"]
        
        
        img_train_folder = img_path +"/"+img_data["dtype"]+"/train"
        img_val_folder = img_path +"/"+img_data["dtype"]+"/val"
        
            
        print("img_train_folder count: ",len(glob(img_train_folder+"/*.png")))
        print("img_val_folder count: ",len(glob(img_val_folder+"/*.png")))
        
        
        label_path = root.format(label_data["path"])
        label_path = label_path + label_data["group_folder"] + "/"+ label_data["drug_n_folder"]
            
        label_train_folder = label_path + "/"+label_data["dtype"]+"/train"
        label_val_folder = label_path+"/"+label_data["dtype"]+"/val"
        
        print("label_train_folder count: ",len(glob(label_train_folder+"/*.txt")))
        print("label_val_folder count: ",len(glob(label_val_folder+"/*.txt")))

In [6]:
item_seqs = [199303108,200400485,201500041,200300406,200301805,201403321   ]
# item_seqs = [199801026]
# album = Albumentation(item_seqs[0])
# album.brightness()


for item_seq in item_seqs:
    album = Albumentation(item_seq)
#     album.train_validation_split()
#     album.flip()
    album.brightness()
    album.getFolderCount()
    print("-----")

item:         item_seq   idx    drug_n        name    group
2515  199303108  2515  K-004378  타이레놀정500mg  TS_3_단일
group_folders:  ['TS_3_단일', 'TL_3_단일']
img_train_folder count:  2592
img_val_folder count:  778
label_train_folder count:  2592
label_val_folder count:  778
-----
item:         item_seq   idx    drug_n  \
2659  200400485  2659  K-014267   

                                                   name     group  
2659  이지엔6프로연질캡슐(덱시부프로펜)(수출명:DAUFENSoftCapsuleSPEEDFEN)  TS_40_단일  
group_folders:  ['TS_40_단일', 'TL_40_단일']
img_train_folder count:  648
img_val_folder count:  196
label_train_folder count:  648
label_val_folder count:  196
-----
item:         item_seq   idx    drug_n          name     group
1224  201500041  1224  K-035073  티지피파모티딘정10mg  TS_74_단일
group_folders:  ['TS_74_단일', 'TL_74_단일']
img_train_folder count:  432
img_val_folder count:  130
label_train_folder count:  432
label_val_folder count:  130
-----
item:        item_seq  idx    drug_n           name    group
109 